In [1]:
from poor_man_lakehouse.spark.builder import retrieve_current_spark_session

spark = retrieve_current_spark_session()


25/08/31 14:42:16 WARN Utils: Your hostname, MacBookPro.homenet.telecomitalia.it resolves to a loopback address: 127.0.0.1; using 192.168.1.81 instead (on interface en0)
25/08/31 14:42:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/graziano/.ivy2/cache
The jars for the packages stored in: /Users/graziano/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.13 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.13 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6bdd8c61-e14e-4de2-9f70-40121de6201e;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.13;1.9.2 in central


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.6/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.13;0.103.3 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.9.2 in central
:: resolution report :: resolve 88ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.9.2 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.13;1.9.2 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.13;0.103.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-6bdd8c61-e14e-4de2-9f70-

In [2]:
spark.sql("use nessie")

DataFrame[]

In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS default").show()


++
||
++
++



In [4]:
spark.sql("SHOW CATALOGS").show()
spark.sql("show schemas").show()
spark.sql("SHOW  tables in default").show()

+-------------+
|      catalog|
+-------------+
|       nessie|
|spark_catalog|
+-------------+

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [5]:
import polars as pl
import datetime
from pyspark.sql import DataFrame

data = pl.DataFrame(
    {
        "datetime": [
            datetime.datetime(2023, 1, 1, 12, 0),
            datetime.datetime(2023, 1, 2, 12, 0),
            datetime.datetime(2023, 1, 3, 12, 0),
        ],
        "symbol": ["AAPL", "GOOGL", "MSFT"],
        "bid": [150.0, 2800.0, 300.0],
        "ask": [151.0, 2805.0, 305.0],
        "details": [
            {"created_by": "user1"},
            {"created_by": "user2"},
            {"created_by": None},
        ],
    },
)

spark_df: DataFrame = spark.createDataFrame(data.to_pandas())


In [6]:
spark_df.write.format("iceberg").mode("overwrite").option(
    "path", "s3a://warehouse/default/prova/"
).saveAsTable("nessie.default.prova")

In [7]:
spark.sql("""SELECT * FROM default.prova""").show()

+-------------------+------+------+------+--------------------+
|           datetime|symbol|   bid|   ask|             details|
+-------------------+------+------+------+--------------------+
|2023-01-01 12:00:00|  AAPL| 150.0| 151.0|{created_by -> us...|
|2023-01-02 12:00:00| GOOGL|2800.0|2805.0|{created_by -> us...|
|2023-01-03 12:00:00|  MSFT| 300.0| 305.0|{created_by -> NULL}|
+-------------------+------+------+------+--------------------+



In [9]:
spark.table("default.people").show()

+---+----------+---------+---+
| id|first_name|last_name|age|
+---+----------+---------+---+
|  5|   Charlie|    Brown| 30|
|  1|      John|      Doe| 28|
| 10|     Henry|    Davis| 38|
|  7|       Eve|   Garcia| 32|
|  3|     Alice|  Johnson| 22|
|  6|     David|    Jones| 25|
|  9|     Grace|      Lee| 27|
|  8|     Frank|   Miller| 29|
|  2|      Jane|    Smith| 34|
|  4|       Bob| Williams| 45|
+---+----------+---------+---+

